# Question to answer 

1. [x] for training data, how many prediction items don't exist in prev_items -> **9% of the sessions**
2. [x] for eval data, how many prediction items don't exist in prev_items from test & eval and all training data? -> **2% of the sessions**
3. [x] Compared to all product data, how many items don't exist in train+eval+test data -> 95%

# Packages

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../../'
sys.path.append(base_dir)
import os
from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer

from tqdm import tqdm


from annoy import AnnoyIndex
import polars as pl
import implicit
import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
debug = False

# eval_cg = False
# negative_sampling = 0.1
if debug:
    SAMPLE_NUM = 10000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model = 'w2v_v3'
nic_model = 'nic'
rank_model_version = 'rank_lgbm'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")

# Data (only target language)

In [3]:


train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'products_train.parquet'), n_rows=SAMPLE_NUM)

# Q1 

In [4]:
unique_prev_items = train_pl.select('prev_items').explode('prev_items').unique().collect()
target_df = train_pl.select('next_item').unique().collect()

prev_item_lst = list(unique_prev_items.to_pandas()['prev_items'].values)
target_item_lst = list(target_df.to_pandas()['next_item'].values)

In [5]:
target_len = len(target_item_lst)
intersect_set = set(prev_item_lst)&set(target_item_lst)
no_present = target_len - len(intersect_set)
print(f"{no_present}/{target_len}")
print(round(no_present/target_len,2))

75567/825613
0.09


# Q2 

In [6]:
unique_prev_items = (
    pl.concat([train_pl.select(pl.concat_list([pl.col('prev_items'), pl.col('next_item')]).alias('prev_items')), 
               eval_pl.select('prev_items')
               , test_pl.select('prev_items')
              ]
         , how='vertical')
        .explode('prev_items').unique()
).collect()
target_df = eval_pl.select('next_item').unique().collect()


prev_item_lst = list(unique_prev_items.to_pandas()['prev_items'].values)
target_item_lst = list(target_df.to_pandas()['next_item'].values)


In [7]:
target_len = len(target_item_lst)
intersect_set = set(prev_item_lst)&set(target_item_lst)
no_present = target_len - len(intersect_set)
print(f"{no_present}/{target_len}")
print(round(no_present/target_len,2))

3810/228044
0.02


# Q3 

In [8]:
# product_pl.head().collect()

In [9]:
all_item = pl.concat([train_pl.select(pl.concat_list([pl.col('prev_items'), pl.col('next_item')]).alias('prev_items'))
                      , eval_pl.select(pl.concat_list([pl.col('prev_items'), pl.col('next_item')]).alias('prev_items'))
                   , test_pl.select('prev_items')
              ]
         , how='vertical').explode('prev_items').unique()

In [10]:
# /all_item.head().collect()

In [11]:
item_num = all_item.collect().shape[0]

In [12]:
all_item_num = product_pl.select('id').unique().collect().shape[0]

In [13]:
print(f"{item_num}/{all_item_num} = {round(item_num/all_item_num,2)}")

1338847/1410675 = 0.95
